<a href="https://colab.research.google.com/github/Lucasscheere/analise_de_credito/blob/main/trademaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, date
import plotly.graph_objs as go
import math as math
import plotly.offline as py
py.init_notebook_mode(connected=False)
import plotly.io as pio
pio.renderers.default = "colab"

In [ ]:
df = pd.read_excel('ROBERTO.xlsx',
                  usecols=['Emissão de NF de produto não formatado', 'Data Movimento não formatado', 'Valor NF de produto', 'Data Vencimento não formatado', 'Valor','Data Pagamento não formatado'],
                  parse_dates=['Emissão de NF de produto não formatado', 'Data Vencimento não formatado', 'Data Movimento não formatado', 'Data Pagamento não formatado'],
                  date_format='%Y-%m-%d'
                   )

In [ ]:
df = df.rename(columns={ #Added assignment to df
    'Emissão de NF de produto não formatado':'Data_compra',
    'Valor NF de produto':'Valor_NF_de_produto',
    'Data Vencimento não formatado':'Data_vencimento',
    'Data Pagamento não formatado':'Data_Pagamento'
    })
df.head()

In [ ]:
data_final = datetime.today()
data_inicial = df.Data_compra.min()

In [ ]:
array_datas = pd.date_range(start=data_inicial, end=data_final, freq = 'D')

In [ ]:
df.loc[df['Data_Pagamento'] == "-", 'Data_Pagamento'] = np.nan

In [ ]:
df = df[df['Data_vencimento'] != "-"]

In [ ]:
df2 = pd.DataFrame(array_datas, columns=['Data'])

In [ ]:
df.Data_compra = pd.to_datetime(df.Data_compra)
df.Data_vencimento = pd.to_datetime(df.Data_vencimento)
df.Data_Pagamento = pd.to_datetime(df.Data_Pagamento)

In [ ]:
def comprasdodia(dia):
    valordacompra = df.loc[df['Data_compra']==dia].Valor.sum()
    return valordacompra

In [ ]:
def pagamentosdodia(dia):
    valordopagamento = df.loc[df['Data_Pagamento']==dia].Valor.sum()
    return valordopagamento

In [ ]:
def vencimentosdodia(dia):
    valordovencimento = df.loc[df['Data_vencimento']==dia].Valor.sum()
    return valordovencimento

In [ ]:
df2.head()

In [ ]:
df2['Compras'] =  df2['Data'].apply(comprasdodia)

In [ ]:
df2['Pagamentos'] =  df2['Data'].apply(pagamentosdodia)

In [ ]:
df2['Vencimentos'] =  df2['Data'].apply(vencimentosdodia)

In [ ]:
df2['compras_acumuladas'] = df2.Compras.cumsum()
df2['vencimentos_acumulados'] = df2.Vencimentos.cumsum()
df2['pagamentos_acumulados'] = df2.Pagamentos.cumsum()

In [ ]:
#Gerando a coluna 'valor_em_aberto', com a diferença entre o que foi comprado e o que foi pago
df2['valor_em_aberto'] = df2['compras_acumuladas'].sub(df2['pagamentos_acumulados'])

#Gerando a coluna 'valor_em_atraso', com a diferença entre o que foi comprado e o que foi pago
df2['valor_em_atraso'] = df2['vencimentos_acumulados'].sub(df2['pagamentos_acumulados'])
df2

In [ ]:
#criando gráfico
linevalues = go.Figure()

linevalues.add_trace(
    go.Scatter(
        x=df2.Data, y=df2.valor_em_aberto,
        mode='lines',
        marker = {'color' : '#0592FF'},
        name='Em aberto'
        )
    )

linevalues.add_trace(
    go.Scatter(
        x=df2.Data,
        y=df2.valor_em_atraso,
        mode='lines',
        marker = {'color' : '#FF494C'},
        name='Em atraso'
        )
    )

#layout do título
linevalues.update_layout(
    title='Valores Em aberto x Em atraso (R$)',
    xaxis={'title': 'Período'},
    yaxis={'title': 'Valores'},
    hovermode="x")

linevalues.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        }
    )

#background
linevalues.layout.plot_bgcolor = '#fff'

#grids
linevalues.update_xaxes(
    showgrid=True, gridwidth=1, gridcolor='lightgray',
    showline=True, linewidth=1, linecolor='black'
)

linevalues.update_yaxes(
    showgrid=True, gridwidth=1, gridcolor='lightgray',
    showline=True, linewidth=1, linecolor='black'
)

#display
linevalues.show()